### Importation des modules

In [ ]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

from datetime import datetime, timedelta
import time
import os

### Configuration des options du navigateur Chrome

In [ ]:

def start_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
       
    driver = webdriver.Chrome(options=options)
    return driver 

### Lancement du navigateur avec les paramètres définis

In [ ]:

driver = start_driver()

### Définition du chemin de sauvegarde des résultats

In [ ]:

output_path = r"C:\Users\elaas\Desktop\Comparateur de prix"
output_file = os.path.join(output_path, "Rome_Tunis_100_Vols.csv")

### Vérification et création du dossier de sortie si nécessaire

In [ ]:

if not os.path.exists(output_path):
    os.makedirs(output_path)

### Modèle d'URL pour récupérer les vols depuis Trip.com

In [ ]:

url_template = (
    "https://fr.trip.com/flights/showfarefirst?"
    "lowpricesource=searchform&triptype=RT&class=Y&quantity=1&childqty=0&babyqty=0"
    "&dcity=rom&acity=tun&ddate={}&rdate={}&dcityName=Rome&acityName=Tunis&locale=fr-FR&curr=EUR"
)

### Définition de la période de recherche des vols

In [ ]:

start_date = datetime(2025, 2, 10)
end_date = datetime(2025, 6, 30) 

dates = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date - start_date).days + 1)]

### Initialisation de la liste pour stocker les données des vols

In [ ]:

all_flights_data = []

### Boucle pour parcourir chaque date et récupérer les vols

In [ ]:

for departure_date in dates:
    if len(all_flights_data) >= 100:
        break

    return_date = (datetime.strptime(departure_date, "%Y-%m-%d") + timedelta(days=3)).strftime("%Y-%m-%d")
    url = url_template.format(departure_date, return_date)
try:
        print(f"Recherche des vols pour le {departure_date}...")
        
        if driver is None or len(driver.window_handles) == 0:
            print("Fenêtre fermée, redémarrage du navigateur...")
            driver = start_driver()
          
        driver.get(url)
        time.sleep(10) 
        
        wait = WebDriverWait(driver, 30)
        flights = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "result-item")))

        print(f"{len(flights)} vols trouvés pour le {departure_date}")

            
for flight in flights:
            if len(all_flights_data) >= 100:  
                break

            try:
                airline = flight.find_element(By.CLASS_NAME, "flights-name").text.strip()
                departure_time = flight.find_elements(By.CSS_SELECTOR, "[data-testid^='flight-time']")[0].text.strip()
                arrival_time = flight.find_elements(By.CSS_SELECTOR, "[data-testid^='flight-time']")[1].text.strip()
                duration = flight.find_element(By.CSS_SELECTOR, "[data-testid='flightInfoDuration']").text.strip()
                price_eur = flight.find_element(By.CSS_SELECTOR, "[data-price]").text.strip().replace("€", "").strip()
      
                all_flights_data.append({
                    "Compagnie": airline,
                    "Pays de départ": "Italie",
                    "Pays de destination": "Tunisie",
                    "Date de départ": departure_date,
                    "Heure de départ": departure_time,
                    "Date de retour": return_date,
                    "Heure d'arrivée": arrival_time,
                    "Durée": duration,
                    "Prix (€)": f"€ {price_eur}",
                })

                print(f"Vol trouvé : {departure_date} → {return_date} - {airline} - €{price_eur}")

            except Exception as e:
                print(f"Erreur d'extraction du vol {departure_date} : {e}")

            except Exception as e:
                   print(f"Problème avec la date {departure_date} : {e}")

### Vérification si des vols ont été collectés

In [ ]:
if len(all_flights_data) > 0:
    print(f"Nombre de vols collectés : {len(all_flights_data)}")
 
    df = pd.DataFrame(all_flights_data, columns=[
        "Compagnie", "Pays de départ", "Pays de destination",
        "Date de départ", "Heure de départ", "Date de retour",
        "Heure d'arrivée", "Durée", "Prix (€)"
    ])


### Sauvegarde des données dans un fichier CSV
    

In [ ]:
df.to_csv(output_file, index=False, encoding="utf-8-sig")

if os.path.exists(output_file):
        print(f"Fichier sauvegardé avec succès : {output_file}")
else:
        print("Échec de la sauvegarde du fichier.")
else:
print("Aucun vol trouvé. Le fichier ne sera pas créé.")

### Fermeture du navigateur à la fin de l'exécution du script

In [ ]:

driver.quit()